In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, *-*"

Installed Packages Microsoft.SemanticKernel, 1.6.3 Microsoft.SemanticKernel.Planners.Handlebars, 1.6.3-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning;


In [3]:
#!import ../Utils/Settings.cs
#!import ../../../Plugins/CustomPlugin/CompanySearchPlugin.cs

In [4]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

In [5]:
Kernel kernel = Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion("Your Azure OpenAI Service gpt-35-turbo-16k Deployment Name", Settings.AOAIEndpoint, Settings.AOAIKey)
            .Build();

In [6]:
var companySearchPluginObj = new CompanySearchPlugin();
var companySearchPlugin = kernel.ImportPluginFromObject(companySearchPluginObj, "CompanySearchPlugin");

In [7]:
var pluginDirectory = Path.Combine("../../..", "plugins");

In [8]:
var writetPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "WriterPlugin"));
var emailPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "EmailPlugin"));
var translatePlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "TranslatePlugin"));

In [9]:
string goal =  """
Write email about  travellings tips based on  getting current weather in Guangzhou and translate it to Chinese
""";

In [10]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

In [11]:
#pragma warning disable SKEXP0060

var originalPlan = await planner.CreatePlanAsync(kernel, goal);

In [12]:
Console.WriteLine(originalPlan);

{{!-- Step 0: Extract key values --}}
{{set "currentCity" "Guangzhou"}}
{{set "desiredLanguage" "Chinese"}}

{{!-- Step 1: Get current weather in Guangzhou --}}
{{set "currentWeather" (CompanySearchPlugin-WeatherSearch city=currentCity)}}

{{!-- Step 2: Translate the weather to desired language --}}
{{set "translatedWeather" (TranslatePlugin-Basic input=currentWeather)}}

{{!-- Step 3: Write email about travel tips --}}
{{set "emailContent" (WriterPlugin-Tips input=translatedWeather language=desiredLanguage)}}

{{!-- Step 4: Print the final result --}}
{{json emailContent}}


In [13]:
originalPlan.ToString()

{{!-- Step 0: Extract key values --}}
{{set "currentCity" "Guangzhou"}}
{{set "desiredLanguage" "Chinese"}}

{{!-- Step 1: Get current weather in Guangzhou --}}
{{set "currentWeather" (CompanySearchPlugin-WeatherSearch city=currentCity)}}

{{!-- Step 2: Translate the weather to desired language --}}
{{set "translatedWeather" (TranslatePlugin-Basic input=currentWeather)}}

{{!-- Step 3: Write email about travel tips --}}
{{set "emailContent" (WriterPlugin-Tips input=translatedWeather language=desiredLanguage)}}

{{!-- Step 4: Print the final result --}}
{{json emailContent}}

In [14]:
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

In [15]:
Console.WriteLine(originalPlanResult);

Based on the weather conditions in Guangzhou, with a temperature of 2 degrees Celsius and rainy, here are some dressing tips:

1. 穿厚外套：在这样的温度下，穿一件厚外套是必要的。选择一件保暖的羽绒服或羊毛大衣，帮助保持身体温暖。

2. 戴帽子和手套：考虑到温度较低，戴一顶保暖的帽子和一双手套可以防止头部和手部的寒冷。

3. 穿长袖衣服和长裤：选择长袖衣服和长裤来保护皮肤免受低温和雨水的侵害。可以选择防水材质的外套和裤子，以防止湿气渗透。

4. 穿雨鞋或防水鞋：由于天气潮湿，穿一双雨鞋或防水鞋是必要的，以防止脚部受潮。

5. 带雨伞：随身携带一把雨伞，以防突然下雨时保持干燥。

请注意，这些建议是基于天气情况的一般建议。具体的穿着需要根据个人的喜好和需求进行调整。
